In [2]:
import cv2 as cv
import tensorflow as tf
import numpy as np
import pandas as pd
import time 
import math 
import mediapipe as mp 

In [3]:
# 1st capture video 
# identify using harcascade

In [3]:
def rescaleFrame(frame, scale =0.75):
    width = int(frame.shape[1]*scale)
    height = int(frame.shape[0]*scale)
    return cv.resize(frame, (width, height), interpolation= cv.INTER_AREA)

In [4]:
# mpHands = mp.solutions.hands # to detect the hands
# hands = mpHands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5) # detect 2 hands, with min confidence of 0.5,, and min tracking confidence of 0.5
# mpDraw = mp.solutions.drawing_utils # to draw key points
# pTime = 0
# cTime = 0

In [5]:
# # to capture video frame by frame using VideoCapture 
# capture = cv.VideoCapture(0)

# while True: # for looping over read object 
#     isTrue, frame = capture.read() # boolean, frame 
#     frame_RGB = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
#     results = hands.process(frame_RGB)
# #     print(results.multi_hand_landmarks)
#     if results.multi_hand_landmarks:
#         for handLms in results.multi_hand_landmarks:
#             for id, lm in enumerate(handLms.landmark):
# #                 print('------------')
# #                 print(id)
#                 h, w, c = frame.shape
#                 cx, cy = int(lm.x*w), int(lm.y*h)
#                 cv.circle(frame, (cx,cy), 3, (255,0,255), cv.FILLED)
                
#             mpDraw.draw_landmarks(frame, handLms, mpHands.HAND_CONNECTIONS)
    
    
    
#     cTime = time.time()
#     fps = 1/ (cTime-pTime)
#     pTime = cTime
#     frame_resize = rescaleFrame(frame_RGB, scale=1)
#     cv.putText(frame_resize,f'FPS:{int(fps)}', (20,70), cv.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
#     cv.imshow('video resized', frame) # output 
    
#     if cv.waitKey(1) & 0xFF==ord('q'): # q as ending key
#         break

# capture.release()
# cv.destroyAllWindows()

In [4]:
palm = [0,5,9,13,17]
def midpoint(lmlist):
    nl_x = []
    nl_y = []
    for i, x, y in lmlist:
        if i in palm:
            nl_x.append(x)
            nl_y.append(y)
    newx = sum(nl_x)//len(nl_x)
    newy = (sum(nl_y)//len(nl_y))
    return newx,newy 

In [5]:
class hand_recognizer():  
    def __init__(self, mode = False, num_hands=1, detection_confidence = 0.5, tracking_confidence = 0.5, model_complexity = 1):
        self.model_complexity = model_complexity
        self.mode = mode
        self.num_hands = num_hands
        self.detection_confidence = detection_confidence
        self.tracking_confidence = tracking_confidence
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.num_hands, self.model_complexity, self.detection_confidence, self.tracking_confidence)
        self.draw = mp.solutions.drawing_utils
        
    def identifier(self,img, draw = True):
        imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.draw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNo = 0, draw = True):
        lmlist = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id, cx, cy])
                if draw:
                    cv.circle(img, (cx, cy), 3, (255, 0, 255), cv.FILLED)
        return lmlist

In [7]:
pTime = 0
cTime = 0
cap = cv.VideoCapture(0)
detector = hand_recognizer()

while True:
    success, img = cap.read()
    img = detector.identifier(cv.flip(img,1))
    lmlist = detector.findPosition(cv.flip(img,1))
    if len(lmlist) != 0:
#         print(lmlist[0])
        x,y = midpoint(lmlist)
        print("x->"+str(x)+" y->"+str(y))

    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv.putText(img, str(int(fps)), (10, 70), cv.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
    cv.imshow("Image", img)
    if cv.waitKey(1) & 0xFF==ord('q'):
        break
cap.release()
cv.destroyAllWindows()

x->520 y->284
x->524 y->285
x->485 y->221
x->478 y->213
x->478 y->213
x->478 y->212
x->486 y->217
x->487 y->219
x->486 y->219
x->485 y->219
x->485 y->219
x->485 y->219
x->484 y->219
x->483 y->227
x->483 y->226
x->484 y->226
x->484 y->226
x->492 y->228
x->496 y->229
x->496 y->229
x->496 y->229
x->496 y->229
x->499 y->231
x->502 y->233
x->502 y->234
x->502 y->234
x->503 y->234
x->504 y->234
x->507 y->237
x->507 y->237
x->507 y->237
x->508 y->243
x->508 y->243
x->512 y->247
x->513 y->248
x->512 y->248
x->512 y->248
x->503 y->261


In [10]:
def run(cap, detector):
    cap = cv.VideoCapture(0)
    detector = hand_recognizer()
    while True:
        success, img = cap.read()
        img = detector.identifier(cv.flip(img,1))
        lmlist = detector.findPosition(cv.flip(img,1))
        if len(lmlist) != 0:
    #         print(lmlist[0])
            x,y = midpoint(lmlist)
            yield x,y 
#             print(x,y)
#             print("x->"+str(x)+" y->"+str(y))
        if cv.waitKey(1) & 0xFF==ord('q'):
            break
        cv.imshow("Image", img)
        cv.waitKey(1)
    cap.release()
    cv.destroyAllWindows()
    

In [11]:
cap = cv.VideoCapture(0)
detector = hand_recognizer()
for val in run(cv.VideoCapture(0), hand_recognizer()):
    print(val)

(427, 285)
(446, 247)
(453, 251)
(454, 251)
(454, 251)
(454, 256)
(455, 256)
(454, 262)
(456, 263)
(456, 263)
(456, 263)
(456, 265)
(456, 265)
(449, 270)
(445, 268)
(444, 267)
(445, 267)
(452, 271)
(452, 270)
(457, 270)
(462, 274)
(463, 274)
(463, 274)
(458, 275)
(458, 275)
(410, 273)
(380, 254)
(378, 254)
(350, 231)
(347, 229)
(319, 210)
(318, 209)
(288, 194)
(277, 186)
(278, 185)
(278, 185)
(278, 179)
(278, 179)
(275, 161)
(256, 149)
(256, 148)
(237, 143)
(237, 141)
(217, 123)
(217, 123)
(196, 112)
(172, 99)
(171, 100)
(171, 100)
(150, 89)
(149, 88)
(134, 85)
(112, 74)
(114, 72)
(85, 53)
(86, 54)
(66, 41)
(56, 46)
(58, 48)
(93, 42)
(107, 42)
(106, 39)
(109, 39)
(121, 43)
(121, 40)
(120, 39)
(124, 37)
(124, 37)
(123, 37)
(121, 42)
(120, 40)
(115, 37)
(116, 37)
(113, 33)
(112, 31)
(114, 26)
(120, 21)
(120, 22)
(120, 21)
(122, 24)
(122, 22)
(120, 21)
(124, 23)
(126, 20)
(125, 19)
(127, 17)
(128, 16)
(127, 18)
(127, 19)
(131, 21)
(129, 4)
(131, 4)
(130, 5)
(129, 4)
(116, -11)
(111, -6)
(

In [36]:

newx = 0
newy = 0
for i, x, y in lmlist:
    if i in palm:
        
        
        print("id>"+str(i)," x->"+str(x)," y-->"+str(y))

id>0  x->474  y-->400
id>5  x->402  y-->273
id>9  x->434  y-->263
id>13  x->464  y-->265
id>17  x->495  y-->276
